In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append("../TinkerEmulator/")
import TinkerEmulator
%matplotlib inline
%load_ext autoreload
%autoreload 2 

# Bias emulator

In [123]:
emu = TinkerEmulator.TinkerEmulator_Bias()

In [124]:
cosmology = [0.0043*0.7**2, 0.286*0.7**2, -1, 1, 3.0, 70, 3.0]#Obh2 Och2 w0 n_s ln10As H0 N_eff
emu.predict_tinker_parameters(cosmology, 0.2)

(4.468028610892009,
 0.28448480666666665,
 1.405716619633635,
 1.3069858166666666,
 0.6898978333333333,
 -1.2134163860189249)

In [11]:
cosmo_pars = np.loadtxt("../data/parameters.txt")
means = np.loadtxt("../data/rotated_means.txt")
variances = np.loadtxt("../data/rotated_variances.txt")
rotation_matrix = np.loadtxt("../data/rotation_matrix.txt")

In [12]:
y =  means-np.mean(means, axis=0)
ymean = np.mean(means, axis=0)
np.savetxt("../data/y.txt", y)
np.savetxt("../data/ymean.txt", ymean)

In [13]:
for item in np.array(emu.lamb):
    print("    {"+",".join([str(i) for i in item])+"},")

    {-7.4479152680153105,-7.464755353609279,-2.5995566848607203,-5.882716288518299,-3.493908035481021,22.358830237417884,2.0321575020315055},
    {-8.239429860610475,13.078292333287708,-1.6453454526932438,24.5500620772674,-2.208502981716459,6.52984173688444,2.702316841472193},
    {-2.3990173934629153,-4.984896153456564,0.6207752789555486,24.763944064926967,6.6630788163129555,6.460551921638751,5.096751274133398},
    {-4.20023013330144,-3.9710113321598386,2.1024385590746855,23.332575129407882,5.148802333959359,10.753336183339533,9.909683275910155},


In [14]:
print(cosmo_pars.shape, means.shape, variances.shape, rotation_matrix.shape)

(40, 7) (40, 4) (40, 4) (4, 4)


In [93]:
def kernel(x,xprime, lamb):
    """
    lamb : d,1
    xprime : n,d
    x m,d
    return 40,1
    """
    result = np.zeros((len(x), len(xprime)))
    for i in range(len(x)):
        for j in range(len(xprime)):
            result[i,j] = np.exp(-1*np.sum((x[i]-xprime[j])**2/(2*(lamb.reshape(-1,1).T)), axis=1))
    return result
def sigma(x, xprime, lamb):
    return kernel(x,xprime, lamb) 

In [16]:
x0 = np.array([0.0043*0.7**2, 0.286*0.7**2, -1, 1, 3.0, 70, 3.0]).reshape(1,-1)
x1 = np.array([0.0043*0.7**2, 0.286*0.7**2, -1, 1, 3.0, 70, 5.0]).reshape(1,-1)
print(emu.GP_list[0].get_matrix(x0,x1))
print(sigma(x0, x1, np.exp(np.array(emu.lamb)[0])))

[[0.76943028]]
[[0.76943028]]


In [17]:
emu.lamb

[array([-7.44791527, -7.46475535, -2.59955668, -5.88271629, -3.49390804,
        22.35883024,  2.0321575 ]),
 array([-8.23942986, 13.07829233, -1.64534545, 24.55006208, -2.20850298,
         6.52984174,  2.70231684]),
 array([-2.39901739, -4.98489615,  0.62077528, 24.76394406,  6.66307882,
         6.46055192,  5.09675127]),
 array([-4.20023013, -3.97101133,  2.10243856, 23.33257513,  5.14880233,
        10.75333618,  9.90968328])]

In [18]:
print(",".join([str(item)for item in [0.0043*0.7**2, 0.286*0.7**2, -1, 1, 3.0, 70, 5.0]]))

0.002107,0.14013999999999996,-1,1,3.0,70,5.0


In [19]:
variances.shape

(40, 4)

In [20]:
def prediction(xprime):
    print(sigma(xprime.T,cosmo_pars,np.exp(np.array(emu.lamb))[0])[0][0])
    result = []
    for i in range(len(emu.lamb)):
        mu = np.mean(means[:,i])
        sigma_xprime_x = sigma(xprime.T,cosmo_pars,np.exp(np.array(emu.lamb))[i])
        print(sigma_xprime_x.shape)
        sigma_x_x = sigma(cosmo_pars,cosmo_pars, np.exp(np.array(emu.lamb)[i]))+(variances[:,i])*np.eye(len(cosmo_pars))
        if i==0:
            print("test")
            print(np.linalg.inv(sigma_x_x)[0], (means[:,i]-mu), np.linalg.inv(sigma_x_x).dot(means[:,i]-mu)[0], sigma_xprime_x[0][0])
            print("#####"*10)
        result.append(mu+sigma_xprime_x.dot(np.linalg.inv(sigma_x_x).dot(means[:,i]-mu)))
        if i==0:
            print(((sigma_x_x)[10,10]))
            print((np.linalg.inv(sigma_x_x)[20,10]))
    print(sigma_x_x.shape)
    print(result, np.mean(means, axis=0))
    return np.dot(rotation_matrix,np.array(result)).flatten()

In [21]:
(means[:,0]-np.mean(means[:,0]))[18]

0.37515414999999996

In [22]:
prediction(np.array([0.0043*0.7**2, 0.286*0.7**2, -1, 1, 3.0, 70, 3.0]).reshape(-1,1))#

0.2386702821875193
(1, 40)
test
[ 5.49668965e+01  8.03580384e-01 -5.80580295e+00  5.72987755e+00
 -1.06162897e+01  3.63960075e-01 -1.14902105e+01 -5.96920925e-01
  5.52670068e-01 -9.21030066e+00  1.07538299e+00 -4.29068099e+00
 -1.87661912e+01 -3.83283219e+00  2.56806517e-01 -2.00247690e+00
  1.61000644e+00  1.82314949e-01 -2.92951514e+00  3.93692966e+00
 -3.53007529e+00 -1.58272169e+01 -5.26731023e+00  3.55959449e+00
  5.48738918e+00  9.92022064e+00 -5.19240972e-01  1.39492729e-02
 -4.72991336e-01  1.62473965e+00 -2.30778852e+00  8.43129140e-01
  2.97261223e+00 -1.99635320e+00  2.06215277e+00  4.43740060e-01
  2.06942155e+00 -8.72816470e+00  3.05497529e+00  5.30030886e+00] [ 0.22772415 -0.28278585  0.32128415  0.17846415  0.40985415 -0.76878285
  0.40140415  0.24942415 -0.39614285  0.31643415 -0.24992585 -0.12223585
  0.26071415  0.08741415 -0.76097985 -0.06336585 -0.09263585 -0.06395585
  0.37515415  0.37989415  0.04260415  0.15116415 -0.46622185  0.97937415
  0.57734415 -0.09004585 

array([ 1.16115021, -0.59625504,  0.55550433,  0.73369924])

In [23]:
emu.emulate(np.array([0.0043*0.7**2, 0.286*0.7**2, -1, 1, 3.0, 70, 3.0]))

array([ 1.16115021, -0.59625504,  0.55550433,  0.73369924])

In [125]:
print(emu.predict_tinker_parameters(np.array([0.0043*0.7**2, 0.286*0.7**2, -1, 1, 3.0, 70, 3.0]), 0.3))

(4.432419358797392, 0.32058653615384625, 1.3586846172228393, 1.347376928846154, 0.728081326923077, -1.0947315110189246)


In [25]:
np.savetxt("../data/log_bias_metric.txt", np.array(emu.lamb))

In [26]:
np.array(emu.lamb).shape

(4, 7)

# hmf emulator

In [99]:
emu = TinkerEmulator.TinkerEmulator_hmf()

In [100]:
emu.emulate(np.array([0.0043*0.7**2, 0.286*0.7**2, -1, 1, 3.0, 70, 3.0]))

array([0.84172204, 0.51280198, 1.28362419, 0.47179058, 0.01403166,
       0.06153028])

In [101]:
emu.predict_tinker_parameters(np.array([0.0043*0.7**2, 0.286*0.7**2, -1, 1, 3.0, 70, 3.0]), 0.3)

(2.5198116919192444,
 0.8454997909106351,
 0.5441108055593771,
 1.3001900391246013)

In [102]:
###### chto's hmf emulator

In [103]:
emu.lamb

[array([11.81913669, -7.8634768 , -1.34890018, 21.80734912, 23.70985666,
        31.2400688 , 26.4886541 ]),
 array([-5.20581365, -5.34716114,  1.34061625, 57.12272516,  2.5464398 ,
        24.42599039,  5.97639444]),
 array([14.05534512, -1.71715583,  0.69466893, 26.86619208,  2.78142873,
        32.3356182 ,  7.27424005]),
 array([26.81685064, 27.92107158,  0.23339985,  6.86203997, 32.135925  ,
        33.32399853, 35.62887807]),
 array([ 0.26265129,  4.38943236,  1.63154295,  4.65099593,  7.66870705,
        31.98362938,  9.30138597]),
 array([15.0169945 , -3.43445901, -0.80167335,  3.29640847, 27.75394662,
        11.31675081, 11.16665272])]

In [111]:
#Parameter 
cosmo_pars = np.load("../data/training_cosmologies.npy")
data = np.load("../data/rotated_MF_parameters.npy")
means = data[:,:,0]
variances = data[:,:,1]**2
rotation_matrix = np.load("../data/rotation_matrix.npy")
y =  means-np.mean(means, axis=0)
ymean = np.mean(means, axis=0)
#emu.lamb

[array([11.81913669, -7.8634768 , -1.34890018, 21.80734912, 23.70985666,
        31.2400688 , 26.4886541 ]),
 array([-5.20581365, -5.34716114,  1.34061625, 57.12272516,  2.5464398 ,
        24.42599039,  5.97639444]),
 array([14.05534512, -1.71715583,  0.69466893, 26.86619208,  2.78142873,
        32.3356182 ,  7.27424005]),
 array([26.81685064, 27.92107158,  0.23339985,  6.86203997, 32.135925  ,
        33.32399853, 35.62887807]),
 array([ 0.26265129,  4.38943236,  1.63154295,  4.65099593,  7.66870705,
        31.98362938,  9.30138597]),
 array([15.0169945 , -3.43445901, -0.80167335,  3.29640847, 27.75394662,
        11.31675081, 11.16665272])]

In [115]:
ymean

array([-0.38523973, -0.23606555, -0.99612066,  0.59285172, -0.4748609 ,
       -1.03035125])

In [119]:
variable = np.array(emu.lamb)
print(variable.shape)
for item in variable:
    print("    {"+",".join([str(i) for i in item])+"},")

(6, 7)
    {11.819136690440798,-7.86347679921291,-1.3489001790458712,21.807349120065773,23.709856659737703,31.240068804362995,26.488654103499925},
    {-5.205813650960261,-5.3471611379606125,1.3406162460465807,57.12272516043868,2.5464397999063526,24.42599039395141,5.976394443911121},
    {14.05534511533479,-1.7171558279243262,0.6946689314297985,26.866192082546736,2.781428732208767,32.33561819944564,7.274240049786457},
    {26.816850637377595,27.921071577528433,0.2333998509953095,6.862039968911631,32.13592499751718,33.323998526091344,35.62887807108435},
    {0.26265129214647404,4.38943235789435,1.6315429514189743,4.650995934050422,7.668707045752652,31.983629382281514,9.301385973549714},
    {15.016994502681213,-3.434459008144309,-0.8016733530916551,3.29640846737543,27.753946617704653,11.316750811512485,11.166652716045194},


In [105]:
def prediction(xprime):
    result = []
    for i in range(len(emu.lamb)):
        mu = np.mean(means[:,i])
        sigma_xprime_x = sigma(xprime.T,cosmo_pars,np.exp(np.array(emu.lamb))[i])
        sigma_x_x = sigma(cosmo_pars,cosmo_pars, np.exp(np.array(emu.lamb)[i]))+(variances[:,i]+(np.mean(np.sqrt(variances[:,i]))**2))*np.eye(len(cosmo_pars))
        result.append(mu+sigma_xprime_x.dot(np.linalg.inv(sigma_x_x).dot(means[:,i]-mu)))
    return np.dot(rotation_matrix,np.array(result)).flatten()

In [120]:
chto_pred = prediction(np.array([0.0043*0.7**2, 0.286*0.7**2, -1, 1, 3.0, 70, 3.0]).reshape(-1,1))
emu_result = emu.emulate(np.array([0.0043*0.7**2, 0.286*0.7**2, -1, 1, 3.0, 70, 3.0]))
np.testing.assert_array_almost_equal(chto_pred, emu_result)
print("pass test")
print(emu_result)

pass test
[0.84172204 0.51280198 1.28362419 0.47179058 0.01403166 0.06153028]


In [122]:
print(emu.predict_tinker_parameters(np.array([0.0043*0.7**2, 0.286*0.7**2, -1, 1, 3.0, 70, 3.0]), 0.3))

(2.5198116919192444, 0.8454997909106351, 0.5441108055593771, 1.3001900391246013)
